In [1]:
%env MMF_DATA_DIR=/home/jupyter/mmf_data

env: MMF_DATA_DIR=/home/jupyter/mmf_data


In [2]:
import logging
import random
import typing

import torch
from mmf.common.registry import registry
from mmf.utils.build import build_config, build_trainer
from mmf.utils.configuration import Configuration
from mmf.utils.distributed import distributed_init, get_rank, infer_init_method
from mmf.utils.env import set_seed, setup_imports
from mmf.utils.flags import flags
#from mmf.utils.general import log_device_names
#from mmf.utils.logger import setup_logger, setup_very_basic_config

In [3]:
from types import SimpleNamespace
args = SimpleNamespace()
args.config_override=None
args.local_rank = None
args.opts = ['config=projects/mmbt/configs/hateful_memes/with_features.yaml', 'run_type=train_val' ,'dataset=hateful_memes', 'model=mmbt']


In [5]:
setup_imports()

In [6]:
configuration = Configuration(args)
configuration.args = args
config = configuration.get_config()
config.start_rank = 0

In [7]:
config.device_id = 0
predict = False

In [8]:
configuration.import_user_dir()
config = configuration.get_config()

if torch.cuda.is_available():
    torch.cuda.set_device(config.device_id)
    torch.cuda.init()


seed = config.training.seed
config.training.seed = set_seed(seed if seed == -1 else seed + get_rank())
registry.register("seed", config.training.seed)

config = build_config(configuration)

# setup_logger(
#     color=config.training.colored_logs, disable=config.training.should_not_log
# )
# logger = logging.getLogger("mmf_cli.run")
# # Log args for debugging purposes
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
logger.info(configuration.args)
logger.info(f"Torch version: {torch.__version__}")
#log_device_names()
logger.info(f"Using seed {config.training.seed}")

trainer = build_trainer(config)
trainer.load()

DEBUG:root:test
INFO:root:namespace(config_override=None, local_rank=None, opts=['config=projects/mmbt/configs/hateful_memes/with_features.yaml', 'run_type=train_val', 'dataset=hateful_memes', 'model=mmbt'])
INFO:root:Torch version: 1.6.0
INFO:root:Using seed 27957511
INFO:mmf.trainers.mmf_trainer:Loading datasets
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/jupyter/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:mmf.trainers.mmf_trainer:Loading model
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connec

In [13]:
config.optimizer

{'type': 'adam_w', 'params': {'lr': 5e-05, 'eps': 1e-08}}

In [8]:
trainer.train()

INFO:mmf.utils.logger:===== Model =====
INFO:mmf.utils.logger:MMBT(
  (model): MMBTForClassification(
    (bert): MMBTBase(
      (mmbt): MMBTModel(
        (transformer): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (drop

KeyboardInterrupt: 

In [9]:
trainer.train??


Signature: trainer.train()
Docstring: <no docstring>
Source:   
    def train(self):
        self.writer.write("===== Model =====")
        self.writer.write(self.model)

        print_model_parameters(self.model)

        if "train" not in self.run_type:
            self.inference()
            return

        should_break = False

        if self.max_epochs is None:
            self.max_epochs = math.inf
        else:
            self.max_updates = math.inf

        self.model.train()
        self.train_timer = Timer()
        self.snapshot_timer = Timer()

        self.profile("Setup Time")

        torch.autograd.set_detect_anomaly(True)
        self.writer.write("Starting training...")

        while self.num_updates < self.max_updates and not should_break:
            self.current_epoch += 1
            registry.register("current_epoch", self.current_epoch)

            # Seed the sampler in case if it is distributed
            self.dataset_loader.seed_sampler("train", self.curr